In [4]:
import nest_asyncio
import asyncio
from bleak import discover
nest_asyncio.apply()

async def run():
    devices = await discover()
    for d in devices:
        print(d)

loop = asyncio.get_event_loop()
loop.run_until_complete(run())

D8:95:3B:9E:D2:D6: PhysBryk
